In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nltk-data/nltk_data/corpora/wordnet/data.adj
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/LICENSE
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/index.adj
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/README
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/verb.exc
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/index.adv
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/index.verb
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/data.verb
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/lexnames
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/data.noun
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/noun.exc
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/index.noun
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/adj.exc
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/index.sense
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/adv.exc
/kaggle/input/nltk-data/nltk_data/corpora/wordnet/cntlist.rev
/kaggle/input/nltk-data/nltk_data/corpora/wordn

In [2]:
import os
import nltk
from nltk.tokenize import word_tokenize

count = 0
for dirname, _, filenames in os.walk('/kaggle/input/cmu-mosi/Raw/Transcript/Segmented'):
    for filename in filenames:
        if count <=10:
            path=os.path.join(dirname, filename)
            count= count +1

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [55]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import os

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Rejoin tokens into a single string
    processed_text = ' '.join(tokens)
    
    return processed_text


def preprocess_files(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".annotprocessed"):
            with open(os.path.join(input_folder, filename), 'r') as file:
                text = file.read()
                processed_text = preprocess_text(text)

                output_filename = os.path.join(output_folder, filename)
                with open(output_filename, 'w') as output_file:
                    output_file.write(processed_text)

# Replace 'input_folder' and 'output_folder' with your actual input and output folder paths
input_folder = '/kaggle/input/cmu-mosi/Raw/Transcript/Segmented'
output_folder = '/kaggle/working/output_folder'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

preprocess_files(input_folder, output_folder)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!pip install resampy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.6 MB/s eta 0:00:0000:0100:01


In [56]:
import os
import numpy as np
!pip install resampy
import resampy
!pip install librosa==0.8.1
import librosa

def extract_mfcc(file_path, n_mfcc=13):
    try:
        # Load audio file
        audio, _ = librosa.load(file_path, res_type='kaiser_fast')

        # Extract MFCCs
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=n_mfcc)

        # Ensure the number of frames is not greater than 20
        if mfccs.shape[1] > 20:
            mfccs = mfccs[:, :20]
        else:
            # Pad the MFCCs to make them consistent in length
            pad_width = 20 - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

        return mfccs

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

def preprocess_files_audio(input_folder_audio, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for file_name in os.listdir(input_folder_audio):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_folder_audio, file_name)
            mfccs = extract_mfcc(file_path)

            if mfccs is not None:
                # Save the processed data
                output_file_path = os.path.join(output_path, f"{os.path.splitext(file_name)[0]}.npy")
                np.save(output_file_path, mfccs)
                
                
                
# Replace 'input_folder' and 'output_folder' with your actual input and output folder paths
input_folder_audio = '/kaggle/input/cmu-mosi/Raw/Audio/WAV_11025'
output_folder_audio = '/kaggle/working/output_folder_audio'

if not os.path.exists(output_folder_audio):
    os.makedirs(output_folder_audio)

preprocess_files_audio(input_folder_audio, output_folder_audio)


# Video Pre-Processing

In [7]:
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model

In [8]:
# Removing Files from the output folder.

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)

In [9]:
videos_dir = '/kaggle/input/cmu-mosi/Raw/Video/Full'
videos = os.listdir(videos_dir)

def extract_frames(video_path, output_folder, frame_interval=1):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Loop through frames and extract
    for i in range(0, frame_count, frame_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            # Save the frame
            frame_filename = os.path.join(output_folder, f"frame_{i}.png")
            cv2.imwrite(frame_filename, frame)

    # Release the video capture object
    cap.release()

    print(f"Frames extracted from {video_path} to {output_folder}")

i = 0
for video in videos:
    i += 1
    video_path = os.path.join(videos_dir, video)
    output_folder = f"output_frames_{i}"
    frame_interval = 10  # Extract every 10th frame
    extract_frames(video_path, output_folder, frame_interval)


Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/2iD-tVS8NPw.mp4 to output_frames_1
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/dq3Nf_lMPnE.mp4 to output_frames_2
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/zhpQhgha_KU.mp4 to output_frames_3
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/fvVhgmXxadc.mp4 to output_frames_4
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/VCslbP0mgZI.mp4 to output_frames_5
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/8qrpnFRGt2A.mp4 to output_frames_6
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/X3j2zQgwYgE.mp4 to output_frames_7
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/6Egk_28TtTM.mp4 to output_frames_8
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/k5Y_838nuGo.mp4 to output_frames_9
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/OtBXNcAL_lE.mp4 to output_frames_10
Frames extracted from /kaggle/input/cmu-mosi/Raw/

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x58d2417ca180] moov atom not found


Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/Ci-AH39fi3Y.mp4 to output_frames_61
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/f9O3YtZ2VfI.mp4 to output_frames_62
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/8OtFthrtaJM.mp4 to output_frames_63
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/jUzDDGyPkXU.mp4 to output_frames_64
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/v0zCBqDeKcE.mp4 to output_frames_65
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/Sqr0AcuoNnk.mp4 to output_frames_66
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/cXypl4FnoZo.mp4 to output_frames_67
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/vyB00TXsimI.mp4 to output_frames_68
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/0h-zjBukYpk.mp4 to output_frames_69
Frames extracted from /kaggle/input/cmu-mosi/Raw/Video/Full/1DmNV9C1hbY.mp4 to output_frames_70
Frames extracted from /kaggle/input/cmu-

In [10]:
def resize_frames(input_folder, output_folder, target_size):
    for frame_file in os.listdir(input_folder):
        frame_path = os.path.join(input_folder, frame_file)

        # Read the frame
        frame = cv2.imread(frame_path)

        # Resize while maintaining the aspect ratio
        resized_frame = cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA)
        normalized_frame = resized_frame/255.0

        # Save the resized frame
        output_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_path, (normalized_frame*255.0).astype(int))

    print(f"Frames resized and normalized and saved to {output_folder}")

# Example usage
input_folders = "/kaggle/working/"
target_size = (224, 224)
for folder in os.listdir(input_folders):
    input_folder = os.path.join(input_folders, folder)
    resize_frames(input_folder, input_folder, target_size)

Frames resized and normalized and saved to /kaggle/working/output_frames_12
Frames resized and normalized and saved to /kaggle/working/output_frames_60
Frames resized and normalized and saved to /kaggle/working/output_frames_84
Frames resized and normalized and saved to /kaggle/working/output_frames_11
Frames resized and normalized and saved to /kaggle/working/output_frames_28
Frames resized and normalized and saved to /kaggle/working/output_frames_26
Frames resized and normalized and saved to /kaggle/working/output_frames_31
Frames resized and normalized and saved to /kaggle/working/output_frames_37
Frames resized and normalized and saved to /kaggle/working/output_frames_77
Frames resized and normalized and saved to /kaggle/working/output_frames_63
Frames resized and normalized and saved to /kaggle/working/output_frames_55
Frames resized and normalized and saved to /kaggle/working/output_frames_22
Frames resized and normalized and saved to /kaggle/working/output_frames_7
Frames resize

In [11]:
# Load pre-trained VGG16 model without classification head
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

58889256/58889256 [==============================] - 0s 0us/step


In [16]:
import keras
print(keras.__version__)

2.12.0


In [15]:
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.1/997.1 kB 4.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 3.0.0 which is incompatible.


In [20]:
!pip list | grep keras

keras                                    3.0.0
keras-core                               0.1.5
keras-cv                                 0.6.1
keras-nlp                                0.6.1
keras-tuner                              1.3.5


In [28]:
!pip install keras==2.6.0
!pip install tensorflow==2.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: keras
    Found existing installation: keras 3.0.0
    Uninstalling keras-3.0.0:
      Successfully uninstalled keras-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.6.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==2.6.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching

In [34]:
print(model.output_shape[1:])

(None, None, 512)


In [39]:
from keras.applications import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications.vgg16 import preprocess_input

# Load pre-trained VGG16 model without classification head
base_model = VGG16(weights='imagenet', include_top=False)

# Choose a layer from which to extract features
output_layer = base_model.get_layer('block5_pool').output

# Create a new model using the selected output layer
model = Model(inputs=base_model.input, outputs=output_layer)

# Define a simple sentiment analysis model on top of the VGG16 features
sentiment_model = Sequential()
sentiment_model.add(GlobalAveragePooling2D(input_shape=model.output_shape[1:]))
sentiment_model.add(Dense(256, activation='relu'))
sentiment_model.add(Dropout(0.5))
sentiment_model.add(Dense(3, activation='softmax'))  # Assuming 3 classes: positive, negative, neutral

# Compile the sentiment analysis model
sentiment_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
sentiment_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 3)                 771       
                                                                 
Total params: 132,099
Trainable params: 132,099
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Load the VGG16 model
sentiment_model = VGG16(weights='imagenet', include_top=False)

def extract_features(frame):
    # Resize the frame to match the expected input shape of the model
    frame = cv2.resize(frame, (224, 224))

    # Expand dimensions to match model input shape
    frame = np.expand_dims(frame, axis=0)

    # Preprocess frame based on VGG16 requirements
    frame = preprocess_input(frame)

    # Extract features using the pre-trained VGG16 model
    features = sentiment_model.predict(frame)

    return features.flatten()

# Example usage
input_folders = "/kaggle/working/"
columns = []
features_df = pd.DataFrame(columns=columns)  # Initialize the DataFrame with columns

for folder in os.listdir(input_folders):
    input_folder = os.path.join(input_folders, folder)
    i = -1  # Reset the counter for each folder
    for frame_file in os.listdir(input_folder):
        i += 1
        columns.append(i)
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path)
        features = extract_features(frame)
        print(f"Extracted Features for {frame_file}, Shape: {features.shape}")
        features_df = pd.concat([features_df, pd.Series(features, name=i)], axis=1)

# Now features_df contains the extracted features from all frames in all folders

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 131ms/step
Extracted Features for frame_5240.png, Shape: (25088,)
1/1 [==============================] - 0s 20ms/step
Extracted Features for frame_1120.png, Shape: (25088,)
1/1 [==============================] - 0s 19ms/step
Extracted Features for frame_700.png, Shape: (25088,)
1/1 [==============================] - 0s 20ms/step
Extracted Features for frame_4520.png, Shape: (25088,)
1/1 [==============================] - 0s 20ms/step
Extracted Features for frame_2660.png, Shape: (25088,)
1/1 [==============================] - 0s 20ms/step
Extracted Features for frame_260.png, Shape: (25088,)
1/1 [===========

In [47]:
features_df 

,0,1,2,3,4,5,6,7,8,9,...,393,394,395,396,397,398,399,400,401,402
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.32042,3.186711,4.51931,6.854855,8.572261,5.462887,6.004403,12.102474,8.629358,10.296458


In [50]:
x= features_df
import pandas as pd


# Remove rows with null values
x_new = x.dropna()


# Print the shape of the cleaned DataFrame
print(x_new.shape)
x


(25088, 27472)


,0,1,2,3,4,5,6,7,8,9,...,393,394,395,396,397,398,399,400,401,402
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.32042,3.186711,4.51931,6.854855,8.572261,5.462887,6.004403,12.102474,8.629358,10.296458


# Training for Text data

In [52]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Flatten

In [104]:
!pip install transformers
!pip install torch

In [105]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
# Assuming you have a list of text samples and corresponding labels
# Replace text_samples and labels with your actual data
import torch

text_data = '/kaggle/working/output_folder'

# Assuming you have labels (sentiment) for your data
labels = ["positive", "negative", "neutral"]

# Tokenize and preprocess the text data
tokenized_inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt', max_length=100)  # Adjust max_length as needed

# Extract input_ids, attention_mask, and labels
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
labels = torch.tensor(labels)  # Convert labels to a torch tensor

ValueError: too many dimensions 'str'

In [109]:
from sklearn.preprocessing import LabelEncoder

# Assuming you have a list of text samples and corresponding labels
# Replace text_samples and labels with your actual data

text_data = '/kaggle/working/output_folder'

# Assuming you have labels (sentiment) for your data
labels = ["positive", "negative", "neutral"]

# Convert text labels to numerical labels
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(labels)

# Tokenize and preprocess the text data
tokenized_inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt', max_length=100)

# Extract input_ids, attention_mask, and labels
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
labels_tensor = torch.tensor(numerical_labels, dtype=torch.long)  # Convert numerical labels to a torch tensor


In [110]:
from torch.utils.data import TensorDataset, DataLoader

# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_mask, labels)

# Create a DataLoader
batch_size = 8  # Adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Assuming you have a binary cross-entropy loss (for binary classification) or categorical cross-entropy loss (for multi-class classification)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed

# Training loop
num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, masks, targets = batch
        outputs = model(inputs, attention_mask=masks)[0]
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        

AttributeError: 'list' object has no attribute 'size'

In [112]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming you have a list of text samples and corresponding labels
# Replace text_samples and labels with your actual data
# text_samples should be a list of text samples
# labels should be a corresponding list of labels

# Example:
# text_samples = ...  # Your text samples
# labels = ...  # Your corresponding labels

# Convert text labels to numerical labels
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(labels)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(numerical_labels)))

# Tokenize and preprocess the text data
tokenized_inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt', max_length=100)

# Extract input_ids, attention_mask, and labels
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
labels_tensor = torch.tensor(numerical_labels, dtype=torch.long)  # Convert numerical labels to a torch tensor

# Split the data into training and validation sets
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels_tensor, test_size=0.2, random_state=42
)

# Create TensorDatasets for training and validation
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

# Create DataLoaders for training and validation
batch_size = 8  # Adjust as needed
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Assuming you have a binary cross-entropy loss (for binary classification) or categorical cross-entropy loss (for multi-class classification)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed

# Training loop
num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, masks, targets = batch
        outputs = model(inputs, attention_mask=masks)[0]
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Found input variables with inconsistent numbers of samples: [1, 1, 3]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import os

# Specify the path to your preprocessed data folder
data_folder_path = '/path/to/your/preprocessed/data'

# Initialize a Tokenizer
tokenizer = Tokenizer()

# Iterate through each file in the folder
for filename in os.listdir(data_folder_path):
    file_path = os.path.join(data_folder_path, filename)
    
    # Read the contents of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text_data = file.read()
        
        # Update the tokenizer with the text data
        tokenizer.fit_on_texts([text_data])

# Calculate the vocabulary size
vocabulary_size = len(tokenizer.word_index) + 1

print(f"Vocabulary Size: {vocabulary_size}")


In [102]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define parameters
embedding_dim = 100  # You can choose the size of the embedding vector
max_sequence_length = 1000  # Assuming a maximum sequence length of 1000 for your input data
num_classes = 10  # Replace with the actual number of classes in your classification task

# Define the text model
text_model = Sequential()

# Add an embedding layer
text_model.add(Embedding(input_dim=1000, output_dim=embedding_dim, input_length=max_sequence_length))

# Add LSTM layers for sequence modeling
text_model.add(LSTM(64, return_sequences=True))
text_model.add(LSTM(32))

# Flatten or add more dense layers as needed
text_model.add(Dense(64, activation='relu'))
text_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
text_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [103]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have a list of text samples and corresponding labels
# Replace text_samples and labels with your actual data
# text_samples should be a list of text samples
# labels should be a corresponding list of labels

# Load your audio data and labels
text_data = "/kaggle/working/output_folder"
labels = ["positive", "negative", "neutral"]


# Split the data into training and validation sets
train_text_samples, val_text_samples, train_labels, val_labels = train_test_split(
    text_samples, labels, test_size=0.2, random_state=42
)

# Tokenize the text samples
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_samples)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_text_samples)
val_sequences = tokenizer.texts_to_sequences(val_text_samples)

# Pad sequences to a fixed length
max_sequence_length = 100  # Replace with your desired sequence length
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Download pre-trained Word2Vec or GloVe embeddings and load them
# Replace 'path_to_embeddings' with the actual path to your pre-trained embeddings file
# For Word2Vec, you might use gensim library
# For GloVe, you might download pre-trained embeddings from the GloVe website

# Example for Word2Vec:
# from gensim.models import Word2Vec
# word2vec_model = Word2Vec.load('path_to_word2vec_model')

# Example for GloVe:
# embeddings_index = {}
# with open('path_to_glove_file.txt', encoding='utf-8') as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs

# Create an embedding matrix
# You might need to adapt this based on your pre-trained em


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (/opt/conda/lib/python3.10/site-packages/keras/preprocessing/sequence.py)

In [ ]:
# Train the text model
text_model.fit(train_text_data, train_labels, validation_data=(val_text_data, val_labels), epochs=num_epochs, batch_size=batch_size)

In [ ]:
# Evaluate the text model on the test set
test_loss, test_accuracy = text_model.evaluate(test_text_data, test_labels)

In [ ]:
# Save the trained text model
text_model.save("text_model.h5")      

# Training for audio

In [84]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM

Training audio using VGGish model

In [88]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGGish  # Example pre-trained model

# Load your audio data and labels
audio_data = "/kaggle/working/output_folder_audio"
labels = ["positive", "negative", "neutral"]

# Preprocess audio data if needed

# Load a pre-trained audio model (e.g., VGGish)
base_model = VGGish(weights='audioset', include_top=False, input_shape=(96, 64, 1))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Build a custom model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))  # Adjust num_classes based on your task

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(audio_data, labels, epochs=10, batch_size=32, validation_split=0.2)


ImportError: cannot import name 'VGGish' from 'tensorflow.keras.applications' (/opt/conda/lib/python3.10/site-packages/keras/api/_v2/keras/applications/__init__.py)

In [95]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

# Load your audio data and labels
audio_data = "/kaggle/working/output_folder_audio"
labels = ["positive", "negative", "neutral"]

# Preprocess audio data if needed

# Load VGGish model from TensorFlow Hub
vggish_url = "https://tfhub.dev/google/vggish/1"
vggish_model = hub.load(vggish_url)

# Build a custom model on top of the pre-trained VGGish model
model = models.Sequential()
model.add(vggish_model)
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))  # Adjust num_classes based on your task

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(audio_data, labels, epochs=10, batch_size=32, validation_split=0.2)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_32/1023914011.py", line 17, in <module>
    model.add(vggish_model)
  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/trackable/base.py", line 205, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
  File "/opt/conda/lib/python3.10/site-packages/keras/engine/sequential.py", line 185, in add
TypeError: The added layer must be an instance of class Layer. Received: layer=<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7c4c21abe260> of type <class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>.

During handling of the above exception, another e

In [97]:
!pip install scikit-learn

In [96]:
# Define the audio model
audio_model = Sequential()
num_classes = 3

# Flatten MFCC features or use LSTM layers for sequence modeling
audio_model.add(Flatten())  # Use this for flattening MFCC features
# Alternatively, use LSTM layers if you have sequential data
# audio_model.add(LSTM(64, return_sequences=True))
# audio_model.add(LSTM(32))

# Add dense layers
audio_model.add(Dense(64, activation='relu'))
audio_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
audio_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
# Train the audio model
audio_model.fit(train_audio_data, train_labels, validation_data=(val_audio_data, val_labels), epochs=num_epochs, batch_size=batch_size)

NameError: name 'train_audio_data' is not defined

In [ ]:
# Evaluate the audio model on the test set
test_loss, test_accuracy = audio_model.evaluate(test_audio_data, test_labels)

In [ ]:
# Save the trained audio model
audio_model.save("audio_model.h5")

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras import layers, Model


# Load and preprocess text data using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Assume audio and video data are numpy arrays
audio_data = ...  # Your audio data
video_data = ...  # Your video data
labels = ...  # Your labels

# Build custom CubeMLP models for audio and video
def build_cube_mlp_model(input_shape):
    # Define your CubeMLP architecture here
    input_layer = layers.Input(shape=input_shape)
    # Add CubeMLP layers as needed
    # Example:
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    return Model(inputs=input_layer, outputs=x)

audio_model = build_cube_mlp_model(input_shape=(audio_data.shape[1],))
video_model = build_cube_mlp_model(input_shape=(video_data.shape[1],))

# Fusion layer (concatenation for simplicity)
fusion_layer = layers.Concatenate()([text_model.output, audio_model.output, video_model.output])

# Final sentiment analysis model
output_layer = layers.Dense(1, activation='sigmoid')(fusion_layer)
final_model = Model(inputs=[text_model.input, audio_model.input, video_model.input], outputs=output_layer)

# Compile the model
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on your multimodal dataset
final_model.fit(
    [tokenizer(text_data, padding=True, truncation=True, return_tensors='tf')['input_ids'], audio_data, video_data],
    labels,
    epochs=5
)

# Evaluate the model
test_text_data = ...  # Your test text data
test_audio_data = ...  # Your test audio data
test_video_data = ...  # Your test video data
test_labels = ...  # Your test labels

accuracy = final_model.evaluate(
    [tokenizer(test_text_data, padding=True, truncation=True, return_tensors='tf')['input_ids'], test_audio_data, test_video_data],
    test_labels
)

print(f'Test Accuracy: {accuracy[1]}')

# Model for training

In [ ]:
import os
import numpy as np


# Function to load and preprocess text data from text files
def load_text_data(folder_path):
    texts = []
    labels = []
    for filename in os.listdir(folder_path):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            # Assuming labels are encoded in the filename or elsewhere in the data
            # Adjust the label extraction logic based on your data
            label = extract_label_from_filename(filename)
            texts.append(text)
            labels.append(label)
    return texts, labels

# Function to load and preprocess audio data from audio files
def load_audio_data(folder_path):
    audio_data = []
    labels = []
    for filename in os.listdir(folder_path):
        # Load and preprocess audio files using a library like librosa or another audio processing library
        # Adjust the audio loading and preprocessing logic based on your data
        audio, label = load_and_preprocess_audio(os.path.join(folder_path, filename))
        audio_data.append(audio)
        labels.append(label)
    return np.array(audio_data), labels

# Function to load and preprocess video data from video files
def load_video_data(folder_path):
    video_data = []
    labels = []
    for filename in os.listdir(folder_path):
        # Load and preprocess video files using a video processing library
        # Adjust the video loading and preprocessing logic based on your data
        video, label = load_and_preprocess_video(os.path.join(folder_path, filename))
        video_data.append(video)
        labels.append(label)
    return np.array(video_data), labels

# Use the functions to load data from the respective folders
text_data, text_labels = load_text_data(os.path.join(output_folder, 'text'))
audio_data, audio_labels = load_audio_data(os.path.join(output_folder_audio, 'audio'))
video_data, video_labels = load_video_data(os.path.join(data_folder, 'video'))

if not os.path.exists(folder_path):
    raise FileNotFoundError(f"Directory not found: {folder_path}")

In [60]:
import os

text_data = "/kaggle/working/output_folder"
audio_data = "/kaggle/working/output_folder_audio"
video_data = "/kaggle/working"

# Get the number of files in each directory
text_file_count = len(os.listdir(text_data))
audio_file_count = len(os.listdir(audio_data))
video_file_count = len(os.listdir(video_data))

print("Text data file count:", text_file_count)
print("Audio data file count:", audio_file_count)
print("Video data file count:", video_file_count)

Text data file count: 93
Audio data file count: 93
Video data file count: 95


In [67]:
!cd /kaggle/working
!mkdir output_folder_video

mkdir: cannot create directory ‘output_folder_video’: File exists


In [ ]:
import os
import shutil

# Set the source directory containing the 93 frame folders
source_directory = '/kaggle/working/'

# Set the destination directory for the new directory containing 93 frame folders
destination_directory = '/kaggle/working/output_folder_video/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# List all folders in the source directory
folders = [folder for folder in os.listdir(source_directory) if os.path.isdir(os.path.join(source_directory, folder))]

# Move each frame folder to the destination directory
for folder in folders:
    folder_path = os.path.join(source_directory, folder)
    destination_path = os.path.join(destination_directory, folder)
    shutil.move(folder_path, destination_path)

print(f"{len(folders)} frame folders moved to {destination_directory}")

In [68]:
import os
import shutil

# Set the source directory containing the frame folders
source_directory = '/kaggle/working/'

# Set the destination directory for the new directory containing frame folders
destination_directory = '/kaggle/working/output_folder_video/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# List all folders in the source directory
folders = [folder for folder in os.listdir(source_directory) if os.path.isdir(os.path.join(source_directory, folder))]

# Filter folders that match the pattern "output_frames_i"
filtered_folders = [folder for folder in folders if folder.startswith('output_frames_') and folder[14:].isdigit()]

# Move each frame folder to the destination directory
for folder in filtered_folders:
    folder_path = os.path.join(source_directory, folder)
    destination_path = os.path.join(destination_directory, folder)
    shutil.move(folder_path, destination_path)

print(f"{len(filtered_folders)} frame folders moved to {destination_directory}")

93 frame folders moved to /kaggle/working/output_folder_video/


In [70]:
import os

text_data = "/kaggle/working/output_folder"
audio_data = "/kaggle/working/output_folder_audio"
video_data = "/kaggle/working/output_folder_video"

# Get the number of files in each directory
text_file_count = len(os.listdir(text_data))
audio_file_count = len(os.listdir(audio_data))
video_file_count = len(os.listdir(video_data))

print("Text data file count:", text_file_count)
print("Audio data file count:", audio_file_count)
print("Video data file count:", video_file_count)

Text data file count: 93
Audio data file count: 93
Video data file count: 93


In [ ]:
import os
import shutil

# Set the source directory path where your video files are
source_directory = '/kaggle/input/your_source_directory'

# Set the name of the new destination directory
new_destination_directory_name = 'your_new_directory'

# Create the new destination directory path
destination_directory = os.path.join('/kaggle/output', new_destination_directory_name)

# Create the new destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Loop through each file in the source directory
for root, dirs, files in os.walk(source_directory):
    for file in files:
        # Check if the file is a video file (you can customize this check based on your file extensions)
        if file.lower().endswith(('.mp4', '.avi', '.mkv', '.mov')):
            # Get the full path of the current file
            file_path = os.path.join(root, file)
            
            # Move the file to the new destination directory
            shutil.move(file_path, destination_directory)

print(f"All video files moved to {destination_directory}")

In [74]:
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras import layers, Model

# Load and preprocess text data using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Assume audio and video data are numpy arrays
# Make sure they have the same number of samples
audio_data = "/kaggle/working/output_folder_audio"
text_data = "/kaggle/working/output_folder"
video_data = "/kaggle/working/output_folder_video"
labels = ["positive", "negative", "neutral"]


# Print the shapes of your data arrays
print(f"Text data shape: {text_data.shape}")
print(f"Audio data shape: {audio_data.shape}")
print(f"Video data shape: {video_data.shape}")

# Split the data into training and testing sets
text_train, text_test, audio_train, audio_test, video_train, video_test, labels_train, labels_test = train_test_split(
    text_data,
    audio_data,
    video_data,
    labels,
    test_size=0.2,  # Adjust the test_size as needed
    random_state=42
)

# Build custom CubeMLP models for audio and video
def build_cube_mlp_model(input_shape):
    # Define your CubeMLP architecture here
    input_layer = layers.Input(shape=input_shape)
    # Add CubeMLP layers as needed
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    return Model(inputs=input_layer, outputs=x)

audio_model = build_cube_mlp_model(input_shape=(audio_data.shape[1],))
video_model = build_cube_mlp_model(input_shape=(video_data.shape[1],))

# Fusion layer (concatenation for simplicity)
fusion_layer = layers.Concatenate()([text_model.output, audio_model.output, video_model.output])

# Final sentiment analysis model
output_layer = layers.Dense(1, activation='sigmoid')(fusion_layer)
final_model = Model(inputs=[text_model.input, audio_model.input, video_model.input], outputs=output_layer)

# Compile the model
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on your multimodal dataset
final_model.fit(
    [text_train, audio_train, video_train],
    labels_train,
    epochs=5
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'str' object has no attribute 'shape'

In [79]:
import os
import numpy as np

# Function to load data from a folder and convert it to a NumPy array
def load_data_from_folder(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Assuming each file contains a single column of numerical data in NumPy format
        file_data = np.load(file_path)
        data.append(file_data)
    return np.array(data)

# Replace 'audio_folder', 'text_folder', and 'frames_folder'
# with the actual paths to your audio, text, and frames dataset folders
audio_folder = '/kaggle/working/output_folder_audio'
text_folder = '/kaggle/working/output_folder'
frames_folder = '/kaggle/working/output_folder_video'

audio_data = load_data_from_folder(audio_folder)
text_data = load_data_from_folder(text_folder)
frames_data = load_data_from_folder(frames_folder)

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [ ]:
# Evaluate the model on the test set
accuracy = final_model.evaluate(
    [text_test, audio_test, video_test],
    labels_test
)

print(f'Test Accuracy: {accuracy[1]}')


# New

In [81]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your preprocessed data
audio_data = '/kaggle/working/output_folder_audio'
text_data = '/kaggle/working/output_folder'
frames_data = '/kaggle/working/output_folder_video'

# Assuming you have labels (sentiment) for your data
labels = ["positive", "negative", "neutral"]

# Perform any additional feature engineering, if needed

# Split the data into training and testing sets
text_train, text_test, audio_train, audio_test, video_train, video_test, labels_train, labels_test = train_test_split(
    text_data, audio_data, video_data, labels, test_size=0.2, random_state=42
)

# Standardize numerical features
scaler = StandardScaler()
text_train = scaler.fit_transform(text_train)
text_test = scaler.transform(text_test)

# Build the Cubic Multimodal LSTM-Perceptron model
text_input = layers.Input(shape=(your_text_feature_shape,))
audio_input = layers.Input(shape=(your_audio_feature_shape,))
video_input = layers.Input(shape=(your_video_feature_shape,))

# Add your desired layers for text, audio, and video modalities
# Example layers for text
text_output = layers.Dense(128, activation='relu')(text_input)

# Example layers for audio
audio_output = layers.LSTM(64)(audio_input)

# Example layers for video
video_output = layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(video_input)
video_output = layers.MaxPooling2D(pool_size=(2, 2))(video_output)
video_output = layers.Flatten()(video_output)

# Combine modalities using Concatenate or any other suitable method
merged = layers.Concatenate()([text_output, audio_output, video_output])

# Add more layers for multimodal processing
merged = layers.Dense(256, activation='relu')(merged)
merged = layers.Dropout(0.5)(merged)

# Output layer
output = layers.Dense(1, activation='sigmoid')(merged)

# Create and compile the model
model = models.Model(inputs=[text_input, audio_input, video_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    [text_train, audio_train, video_train],
    labels_train,
    epochs=10,  # Adjust as needed
    batch_size=32,  # Adjust as needed
    validation_data=([text_test, audio_test, video_test], labels_test)
)

# Evaluate the model
accuracy = model.evaluate([text_test, audio_test, video_test], labels_test)[1]
print(f"Accuracy: {accuracy}")

# Perform hyperparameter tuning using tools like GridSearchCV or RandomizedSearchCV
# Adjust the model architecture, learning rate, batch size, etc. to find the best combination


ValueError: Found input variables with inconsistent numbers of samples: [29, 35, 35, 3]